In [4]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


#Reading Data from CSV Files
temperature_readings = pd.read_csv('data/data.csv') #original dataset for temp readings 
temperature_readings = temperature_readings.interpolate(method='linear')#fill in null values


weather_readings = pd.read_csv('data/weatherData.csv') #orignal dataset for weather readings

# Converting date and time in temperature_readings to UNIX timestamp for comparision 
temperature_readings['datetime_str'] = temperature_readings['date'] + ' ' + temperature_readings['time']
temperature_readings['datetime'] = temperature_readings['datetime_str'].apply(lambda x: datetime.strptime(x, "%a %b %d %Y %I:%M:%S %p"))
temperature_readings['unix_timestamp'] = temperature_readings['datetime'].apply(lambda x: int(x.timestamp()))

# Converting date and time in weather_readings to UNIX timestamp for comparision 
weather_readings['datetime_str'] = weather_readings['date'] + ' ' + weather_readings['time']
weather_readings['datetime'] = weather_readings['datetime_str'].apply(lambda x: datetime.strptime(x, "%a %b %d %Y %I:%M:%S %p"))
weather_readings['unix_timestamp'] = weather_readings['datetime'].apply(lambda x: int(x.timestamp()))

weather_result_col = [col for col in weather_readings.columns if "result" in col.lower()]

# Merging both data for it to be on the same time
merged_data = pd.merge_asof(
    temperature_readings,  # Left DataFrame
    weather_readings[['unix_timestamp']+weather_result_col],      # Right DataFrame
    on='unix_timestamp',   # Key column
    direction='nearest'    # Match the nearest time
)

datetime_string = merged_data['date'] + " " + merged_data["time"]
merged_data["ISO_formatted_datetime"] = pd.to_datetime(
    datetime_string,
    format="%a %b %d %Y %I:%M:%S %p"
)


# #Columns for lorWan Sensors
temperature_col = [
    col for col in merged_data.columns 
    if "lorawan_readings" in col.lower() and "temperature" in col.lower()
]

humidity_col = [
    col for col in merged_data.columns 
    if "humidity" in col.lower() and "lorawan_readings" in col.lower()
]

co2_col = [
    col for col in merged_data.columns 
    if "co2" in col.lower() and "lorawan_readings" in col.lower()
]

sensors_to_keep = ["Sensor_1", "Sensor_3", "Sensor_6"]
sensors_col = [col for col in merged_data.columns if any(sensor in col for sensor in sensors_to_keep)]

weather_cols_to_keep = ["weather_status","weather_temp","weather_humidity"]
weather_col = [col for col in merged_data.columns if any(weathercol in col for weathercol in weather_cols_to_keep)]

#adding avg temp humid and co2
merged_data['avg_temperature'] = merged_data[temperature_col].mean(axis=1)
merged_data['avg_humidity'] = merged_data[humidity_col].mean(axis=1)
merged_data['avg_co2'] = merged_data[co2_col].mean(axis=1)

avg_col = [
    col for col in merged_data.columns
    if "avg" in col.lower()
]

#Energy(power,energy,current) Data
energy_data = merged_data[["ISO_formatted_datetime"]+ sensors_col]
energy_data.columns = energy_data.columns.str.replace(
    r"Energy_Readings.Sensor_1\.(Current|Energy|Power)", "compressor_\\1", regex=True
).str.replace(
    r"Energy_Readings.Sensor_3\.(Current|Energy|Power)", "fancoil_1_\\1", regex=True
).str.replace(
    r"Energy_Readings.Sensor_6\.(Current|Energy|Power)", "fancoil_2_\\1", regex=True
)

#indoor Data
indoor_data = merged_data[["ISO_formatted_datetime"] + temperature_col + humidity_col + co2_col + avg_col]


#Weather data
weather_data = merged_data[["ISO_formatted_datetime"]+ weather_col]
weather_data.columns = weather_data.columns.str.replace(
    r"result.weather_status", "weather_status", regex=True
).str.replace(
    r"result.weather_temp", "weather_temp", regex=True
).str.replace(
    r"result.weather_humidity", "weather_humidity", regex=True
)


#merging all the needed data
energy_indoor_merged = pd.merge(energy_data, indoor_data, on='ISO_formatted_datetime', how='inner')
final_merged_data = pd.merge(energy_indoor_merged, weather_data, on='ISO_formatted_datetime', how='inner')

final_merged_data['total_energy'] = (
    final_merged_data['compressor_Energy'] +
    final_merged_data['fancoil_1_Energy'] +
    final_merged_data['fancoil_2_Energy']
)

final_merged_data['total_power'] = (
    final_merged_data['compressor_Power'] +
    final_merged_data['fancoil_1_Power'] +
    final_merged_data['fancoil_2_Power']
)

final_merged_data['total_current'] = (
    final_merged_data['compressor_Current'] +
    final_merged_data['fancoil_1_Current'] +
    final_merged_data['fancoil_2_Current']
)
final_merged_data['hour'] = pd.to_datetime(final_merged_data['ISO_formatted_datetime']).dt.hour
final_merged_data['day_of_week'] = pd.to_datetime(final_merged_data['ISO_formatted_datetime']).dt.dayofweek

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

C:\Users\yy\AppData\Local\Temp\ipykernel_9692\850930544.py:20: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  temperature_readings = temperature_readings.interpolate(method='linear')#fill in null values


In [5]:
def create_target_col(df):
    comfort_condition = (
        (df['avg_co2'] < 1000) &  # CO2 level
        (40 <= df['avg_humidity']) & (df['avg_humidity'] <= 60)
    )
    energy_eff_condtion =(
        (df['total_power'] < 10) & 
        (df['total_current'] < 13)
    )

    comfortable_temp = df.loc[comfort_condition & energy_eff_condtion, 'avg_temperature'].median()

    power_weight = 1 - (df['total_power'] / df['total_power'].max())
    current_weight = 1 - (df['total_current'] / df['total_current'].max())
        
    '''
    Target temperature calculation:
    1. If comfort conditions are met, use the actual temperature
    2. Otherwise, calculate a weighted temperature considering:
        70% of the comfortable mean temperature
        30% of the actual temperature
        Adjusted by power and current consumption weights
    '''
    target = np.where(
        comfort_condition, 
        df['avg_temperature'], 
        0.7 * comfortable_temp + 0.3 * df['avg_temperature'] * (power_weight + current_weight) / 2
    )
    return target
    
     
target = create_target_col(final_merged_data)
final_merged_data['optimal_temp'] = target


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
import os
from tensorflow.keras.optimizers import Adam


# Load your data into `final_merged_data`
# final_merged_data = pd.read_csv("your_data.csv")

# Relevant columns for input
input_features = [
    'avg_temperature', 'avg_humidity', 'avg_co2',
    'weather_temp', 'weather_humidity', 'total_energy', 
    'total_power', 'total_current', 'hour', 'day_of_week'
]
categorical_features = ['weather_status']

# Preprocessing
X = final_merged_data[input_features + categorical_features]
y = final_merged_data['optimal_temp']  # Replace with the actual column name for the target

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), input_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

X = preprocessor.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model-building function for KerasTuner
def build_model(hp):
    model = Sequential()
    
    # Input layer
    model.add(Dense(
        units=hp.Int('units_input', min_value=16, max_value=64, step=16),
        activation='relu',
        kernel_regularizer=l2(hp.Float('l2_input', 0.01, 0.1, step=0.01)),
        input_shape=(X_train.shape[1],)
    ))

        
    model.add(Dropout(rate=hp.Float('dropout_input', 0.0, 0.5, step=0.1)))
    
    # Hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):  # Try 1 to 3 hidden layers
        model.add(Dense(
            units=hp.Int(f'units_{i}', min_value=16, max_value=64, step=16),
            activation='relu',
            kernel_regularizer=l2(hp.Float('l2_hidden', 0.01, 0.1, step=0.01))
        ))
    
    # Output layer
    model.add(Dense(1, activation='linear'))
    
    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4]))

    
    # Compile the model
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['mae']
    )
    return model

# Initialize KerasTuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',  # Minimize validation loss
    max_trials=100,         # Number of hyperparameter combinations
    executions_per_trial=2,  # Number of runs per combination for stability
    directory='hyperparam_tuning',
    project_name='optimal_temp'
)

# Train models using hyperparameter search
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

tuner.search(
    X_train, y_train,
    epochs=500,
    validation_split=0.3,
    batch_size=8,
    callbacks=[early_stopping]
)

# Retrieve all trial results
all_trials = []
for trial in tuner.oracle.get_best_trials(num_trials=20):
    trial_result = {
        'trial_id': trial.trial_id,
        'units_input': trial.hyperparameters.get('units_input'),
        'num_layers': trial.hyperparameters.get('num_layers'),
        'l2_input': trial.hyperparameters.get('l2_input'),
        'learning_rate': trial.hyperparameters.get('learning_rate'),
        'val_loss': trial.metrics.get_best_value('val_loss'),
        'val_mae': trial.metrics.get_best_value('val_mae'),
    }
    # Add hidden layer details
    for i in range(trial.hyperparameters.get('num_layers')):
        trial_result[f'units_layer_{i}'] = trial.hyperparameters.get(f'units_{i}')
    all_trials.append(trial_result)

# Convert to DataFrame and save to CSV
all_trials_df = pd.DataFrame(all_trials)
output_dir = "results"
os.makedirs(output_dir, exist_ok=True)
all_trials_df.to_csv(os.path.join(output_dir, "all_hyperparameter_results.csv"), index=False)

print("All hyperparameter results saved to all_hyperparameter_results.csv")


Trial 99 Complete [00h 03m 22s]
val_loss: 0.33106473088264465

Best val_loss So Far: 0.16075565665960312
Total elapsed time: 02h 55m 50s

Search: Running Trial #100

Value             |Best Value So Far |Hyperparameter
16                |32                |units_input
0.01              |0.05              |l2_input
0.3               |0.3               |dropout_input
1                 |1                 |num_layers
48                |64                |units_0
0.1               |0.01              |l2_hidden
0.001             |0.001             |learning_rate
48                |48                |units_1
32                |64                |units_2

Epoch 1/500


C:\Users\yy\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 568.7732 - mae: 23.6605 - val_loss: 178.0321 - val_mae: 12.7818
Epoch 2/500
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 110.0398 - mae: 8.8124 - val_loss: 19.2909 - val_mae: 2.9432
Epoch 3/500
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 45.6449 - mae: 5.1596 - val_loss: 14.3598 - val_mae: 2.3887
Epoch 4/500
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 39.2407 - mae: 4.7648 - val_loss: 11.0866 - val_mae: 2.0696
Epoch 5/500
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 32.9192 - mae: 4.3028 - val_loss: 10.0496 - val_mae: 1.9804
Epoch 6/500
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30.4544 - mae: 4.2472 - val_loss: 7.8751 - val_mae: 1.6607
Epoch 7/500
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 32.9010 - mae: 4.3943 - val_loss: 7.7125 - val_mae: 1.7376
Epoch 8/500
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30.9654 - mae: 4.2258 - val_loss: 6.2804 - val_mae: 1.4619
Epoch 9/500
  1/127 ━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters for reference
print("Best Hyperparameters:")
print(best_hps.values)

In [ ]:
# Rebuild the model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)

# Train the model on the full training data
history = best_model.fit(
    X_train, y_train,
    validation_split=0.3,
    epochs=500,
    batch_size=8,
    callbacks=[early_stopping],
    verbose=1
)

best_model.save('best_trained_model.h5')  # Save in HDF5 format

